In [1]:
import pandas as pd
import requests, warnings
import seasonal_plot
from datetime import datetime as dt
from dateutil import relativedelta

In [2]:
def ag_get_data(query):
    warnings.simplefilter('ignore', requests.packages.urllib3.exceptions.InsecureRequestWarning)
    params = {"Database": "OIL", "Query": query}
    resp = requests.post('https://TST-QDEV-AP1.petroineos.local:5001/genericdata/Fetch/', json=params, verify=False)
    json_result = resp.json()
    df = pd.DataFrame(json_result)
    return df

In [3]:
# pdate = ag_get_data("select max(pdate) from oil.Refinery.MaintenanceDaily")
last_week = (dt.today() - relativedelta.relativedelta(days=7)).strftime('%Y%m%d')

In [4]:
last_week

'20221024'

In [5]:
tar_qry = """
select lst.PLANT_ID, d.UNIT_ID, d.EVENT_ID,d.EVENT_PDATE, d.OUTAGE_DATE, d.CAP_OFFLINE, lst.EVENT_TYPE from oil.Refinery.MaintenanceDaily d
left join oil.Refinery.RefineryMaintenanceBestLatest as lst
on d.EVENT_ID=lst.EVENT_ID
where d.PDate = (select max(pdate) from oil.Refinery.MaintenanceDaily)
"""

prev_tar_qry = f"""
select lst.PLANT_ID, d.UNIT_ID, d.EVENT_ID,d.EVENT_PDATE, d.OUTAGE_DATE, d.CAP_OFFLINE, lst.EVENT_TYPE from oil.Refinery.MaintenanceDaily d
left join oil.Refinery.RefineryMaintenanceBestLatest as lst
on d.EVENT_ID=lst.EVENT_ID
where d.PDate = '{last_week}'
"""

In [6]:
prev_tar_qry

"\nselect lst.PLANT_ID, d.UNIT_ID, d.EVENT_ID,d.EVENT_PDATE, d.OUTAGE_DATE, d.CAP_OFFLINE, lst.EVENT_TYPE from oil.Refinery.MaintenanceDaily d\nleft join oil.Refinery.RefineryMaintenanceBestLatest as lst\non d.EVENT_ID=lst.EVENT_ID\nwhere d.PDate = '20221024'\n"

In [7]:
unit_qry = "select UTYPE_DESC,UNIT_GROUP, COUNTRY, OWNER_NAME, PLANT_NAME, UNIT_ID from oil.Refinery.RefineryUnit"

In [8]:
unit_mapping = ag_get_data(unit_qry)
new_unit_mapping = pd.read_csv('Global Refining Unit Profiles.csv')
short_unit_mapping = new_unit_mapping[['UNIT_TYPE','P_COUNTRY','OWNER_NAME','PLANT_NAME','UNIT_ID']]
short_unit_mapping['UNIT_ID'] = short_unit_mapping['UNIT_ID'].astype(str)

C:\conda\envs\lng\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [9]:
region_mapping = pd.read_csv('tar_region_mapping.csv')
region_mapping_dedup = region_mapping[['Tar_Region','My Region','PlantID']].copy().drop_duplicates(keep='first')
region_mapping_dedup['PlantID'] = region_mapping_dedup['PlantID'].astype(str)

In [10]:
raw_tars = ag_get_data(tar_qry)
old_tars = ag_get_data(prev_tar_qry)

### ------------------ SHORT SUBSET TEST-------------------------

In [11]:
planned_raw = raw_tars[raw_tars['EVENT_TYPE']=='Planned'].copy()
unplanned_raw = raw_tars[raw_tars['EVENT_TYPE']=='Unplanned'].copy()

In [12]:
# last pdate outages
last_planned_raw = old_tars[old_tars['EVENT_TYPE']=='Planned'].copy()
last_unplanned_raw = old_tars[old_tars['EVENT_TYPE']=='Unplanned'].copy()

In [13]:
# def to_monthly(df):
#     df_new = df.set_index(pd.to_datetime(df['OUTAGE_DATE'])).groupby(['PLANT_ID','UNIT_ID']).resample('MS').mean()
#     return df_new

def create_index():
    index_df = pd.DataFrame(pd.date_range(start='2017-01-01', end='2023-12-31', freq='d'))
    index_df.rename(columns={0:'OUTAGE_DATE'}, inplace=True)
    index_df.set_index('OUTAGE_DATE', inplace=True)
    return index_df

def reindex_monthly(df):
    index = create_index()
    grouped = df.sort_values('EVENT_PDATE', ascending=False).groupby(['PLANT_ID', 'UNIT_ID','OUTAGE_DATE']).max()
    reindexed = grouped['CAP_OFFLINE'].unstack([0,1]).merge(index, how='right', left_index=True, right_index=True).fillna(0)
    monthly_pivot = reindexed.resample('MS').mean()
    flat = monthly_pivot.stack([0,1]).reset_index()
    flat.rename(columns={'level_0':'OUTAGE_DATE', 0:'CAP_OFFLINE'}, inplace=True)
    flat.set_index(['PLANT_ID','UNIT_ID','OUTAGE_DATE'], inplace=True)
    flat.sort_index(inplace=True)
    return flat

def unit_name_mapping(df):
    df_mapped = df.reset_index().merge(unit_mapping, how='outer', left_on='UNIT_ID', right_on='UNIT_ID')
    return df_mapped

def remove_nulls(df):
    df_new = df[df['PLANT_NAME'].notnull()].copy()
    return df_new

def get_plant_id(df):
    df_mapped = df.reset_index().merge(region_mapping_dedup, how='left', left_on='PLANT_ID', right_on='PlantID').drop(columns=['index', 'PlantID'])
    return df_mapped

def group(df, col):
    df_c = df.groupby([col, 'UTYPE_DESC','OUTAGE_DATE']).sum()
    return df_c

def kbd(df):
    df_new = df/1000
    return df_new.astype(int)

In [14]:
planned_m = reindex_monthly(planned_raw)
unplanned_m = reindex_monthly(unplanned_raw)
full_m = reindex_monthly(raw_tars)

C:\conda\envs\lng\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
C:\conda\envs\lng\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
C:\conda\envs\lng\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [15]:
planned = get_plant_id(remove_nulls(unit_name_mapping(planned_m)))
unplanned = get_plant_id(remove_nulls(unit_name_mapping(unplanned_m)))
total = get_plant_id(remove_nulls(unit_name_mapping(full_m)))

In [16]:
# last pdate outages
last_planned = get_plant_id(remove_nulls(unit_name_mapping(reindex_monthly(last_planned_raw))))
last_unplanned = get_plant_id(remove_nulls(unit_name_mapping(reindex_monthly(last_unplanned_raw))))
last_total = get_plant_id(remove_nulls(unit_name_mapping(reindex_monthly(old_tars))))

C:\conda\envs\lng\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
C:\conda\envs\lng\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
C:\conda\envs\lng\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [ ]:
len(planned)

In [ ]:
len(unplanned)

### write to excel

In [ ]:
# writer = pd.ExcelWriter('TARs.xlsx', engine='xlsxwriter')
# # kbd(group(unplanned, 'My Region')).to_excel(writer, sheet_name='PQ_RefUnit')
# kbd(group(planned, 'COUNTRY')).reset_index().to_excel(writer, sheet_name='PQ_CtryPlan', index=False)
# kbd(group(planned, 'My Region')).reset_index().to_excel(writer, sheet_name='PQ_RegPlan', index=False)
# kbd(group(unplanned, 'COUNTRY')).reset_index().to_excel(writer, sheet_name='PQ_CtryUnp', index=False)
# kbd(group(unplanned, 'My Region')).reset_index().to_excel(writer, sheet_name='PQ_RegUnp', index=False)
# writer.save()
# # kbd(group(unplanned, 'My Region'))

## pivoting data

In [17]:
table_cols = ['Tar_Region','COUNTRY', 'OWNER_NAME', 'PLANT_NAME','UNIT_GROUP','UTYPE_DESC','UNIT_ID']
old_cols = ['My Region','COUNTRY', 'OWNER_NAME', 'PLANT_NAME','UNIT_GROUP','UTYPE_DESC','UNIT_ID']
# new_cols = ['P_COUNTRY', 'OWNER_NAME', 'PLANT_NAME','UNIT_TYPE','UNIT_ID']
sort_list=['North America','C & S America','Europe','FSU','Africa','AG','Asia Pacific', 'World']

In [18]:
def filter_df(df, value):
    df = df[(df['OUTAGE_DATE']>='2021-01-01') & (df['OUTAGE_DATE']<='2023-06-30')]
    return df[df['UTYPE_DESC']==value]

def pivot(df, cols):
    df_pivot = df.pivot(index=cols,columns='OUTAGE_DATE',values='CAP_OFFLINE').fillna(0)
    return df_pivot.sort_index()

def group_region(df):
    df_g = df.groupby(level=0).sum()
    df_g.loc['World'] = df_g.sum(axis=0)
    return df_g.loc[sort_list]

In [19]:
cdu_planned = filter_df(planned, 'Atmospheric Distillation')
cdu_unplanned = filter_df(unplanned, 'Atmospheric Distillation')
cdu_total = filter_df(total, 'Atmospheric Distillation')

In [20]:
# last pdate outages
last_cdu_total = filter_df(last_total, 'Atmospheric Distillation')

In [21]:
planned_pivot = kbd(pivot(cdu_planned, table_cols))
unplanned_pivot = kbd(pivot(cdu_unplanned, table_cols))

In [ ]:
# writer = pd.ExcelWriter('regional_tar_table.xlsx', engine='xlsxwriter')
# group_region(planned_pivot).to_excel(writer, sheet_name='planned')
# group_region(unplanned_pivot).to_excel(writer, sheet_name='unplanned')
# writer.save()

## by country planned

In [ ]:
planned_pivot.head()

In [ ]:
asia = planned_pivot.loc['Asia Pacific']
asia_un = unplanned_pivot.loc['Asia Pacific']

In [ ]:
asia_countries = asia.groupby(level=0).sum()
asia_countries = asia_un.groupby(level=0).sum()

In [ ]:
asia_countries.T.plot(figsize=(20,10))

In [ ]:
# asia_countries.T.to_csv('planned_asia_outages.csv')

### regional table

In [23]:
def filter_df_unit(df, value):
    df = df[(df['OUTAGE_DATE']>='2022-07-01') & (df['OUTAGE_DATE']<='2023-03-31')]
    return df[df['My Region']==value]

def filter_date(df):
    df = df[(df['OUTAGE_DATE']>='2022-07-01') & (df['OUTAGE_DATE']<='2023-03-31')]
    return df

def group_refinery(df):
    df_g = df.groupby(level=[0,1,3]).sum()
    return df_g

In [20]:
nwe = (filter_df_unit(cdu_total, 'NWE'))
med = (filter_df_unit(cdu_total, 'Med'))

In [21]:
nwe_pivot = group_refinery(kbd(pivot(nwe, old_cols)))
med_pivot = group_refinery(kbd(pivot(med, old_cols)))

In [ ]:
# nwe_pivot.style.background_gradient(cmap='Reds')

### delta vs previous

In [38]:
last_nwe = (filter_df_unit(last_cdu_total, 'NWE'))
last_med = (filter_df_unit(last_cdu_total, 'Med'))
last_nwe_pivot = group_refinery(kbd(pivot(last_nwe, old_cols)))
last_med_pivot = group_refinery(kbd(pivot(last_med, old_cols)))
delta_nwe = nwe_pivot - last_nwe_pivot
delta_med = med_pivot - last_med_pivot

In [24]:
# total CDU outages delta
total = group_refinery(kbd(pivot(filter_date(cdu_total), table_cols)))
prev_total = group_refinery(kbd(pivot(filter_date(last_cdu_total), table_cols)))
delta_world = total - prev_total

In [ ]:
dt.today()

In [25]:
writer = pd.ExcelWriter(f'TAR_changes_{dt.today().date()}.xlsx', engine='xlsxwriter')
delta_world.loc['Europe'].to_excel(writer, sheet_name='delta')
total.loc['Europe'].to_excel(writer, sheet_name='current')
prev_total.loc['Europe'].to_excel(writer, sheet_name='previous')
writer.save()

In [55]:
delta_world.loc['Europe'].to_csv('delta_europe.csv')

In [59]:
total.loc['Europe'].to_csv('europe.csv')

In [60]:
prev_total.loc['Europe'].to_csv('prev_europe.csv')

### seasonal plot

In [ ]:
seasonal_plot()

In [ ]:
nwe_ts = planned.groupby(['My Region','OUTAGE_DATE']).sum().loc['NWE']

## ---------------------- drawing table ---------------------

In [ ]:
# regional_planned.style.background_gradient(cmap='red')

In [ ]:
# regional_planned.style.background_gradient(cmap='Reds')

In [ ]:
regional_planned = group_region(planned_pivot)

In [ ]:
def test_formating(df):
    df.columns = df.columns.strftime('%b-%y')
    styler = df.style.set_table_styles([
    {'selector': 'th',
     'props': [
         ('border-style', 'solid'),
         ('border-color', 'black'),
         ('vertical-align', 'top'),
         ('font-size', '10pt'),
         ('font-family', 'sans-serif'),
         ('background-color', '#edeeef'), ('border-width', '1px'),
         ('border-collapse', 'collapse'), 
         ('text-align', 'center'), ('width', '100px')]}]).set_properties(
    **{'background-color': 'white', 'font-size': '10pt', 'border-color': 'black', 'border-style': 'solid','font-family':'sans-serif',
       'border-width': '1px', 'border-collapse': 'collapse', 'text-align': 'center'}).background_gradient(cmap='Reds')
    return styler

In [ ]:
styled = test_formating(regional_planned)

In [ ]:
def formatter(df):
#     df.columns = df.columns.strftime('%b-%y')
#     styler = df.style.background_gradient(cmap='Reds')
#     styler = styler.format(**{'header':'8pt'})
#     styler = styler.set_properties(**{'font-size': '10pt','font': 'calibri','header':'10pt', 'index':'10pt'})
    styler = df.style.set_table_styles([{
        'selector': 'tr:nth-child(even)', 'props': [('background-color', '#f2f2f2')],
    }])
    
    return styler

In [ ]:
styled = formatter(regional_planned)

In [ ]:
# formatting columns headers
# df.style.format('{:.0f}').hide([('Random', 'Tumour'), ('Random', 'Non-Tumour')], axis="columns")

In [ ]:
styled

In [ ]:
regional_planned.columns.strftime('%b-%y')

In [ ]:
type(styled)

In [ ]:
styled

## ---------------------- email ---------------------

In [ ]:
import ag_email_utils
import pandas as pd

In [ ]:
from email.mime.application import MIMEApplication
from exchangelib import DELEGATE, Account, Credentials, Message, Mailbox, HTMLBody, Configuration, FileAttachment
import os
import email
import ag_encryption as ae
import email.mime.application

In [ ]:
import ag_email_utils
from email.mime.application import MIMEApplication
from exchangelib import DELEGATE, Account, Credentials, Message, Mailbox, HTMLBody, Configuration, FileAttachment
import os
import email
import ag_encryption as ae
import email.mime.application

In [ ]:
email_from='syed.ahmad@petroineos.com'
emails = 'syed.ahmad@petroineos.com'
sub='regional'
body=styled.render()

email_sender = ag_email_utils.AgEmailHelper()
email_sender.send_mail_with_embedded_images(email_from, emails, sub, body)

##  missing records

In [ ]:
planned[planned['PlantID'].isna()]

In [ ]:
## unique missing unit ID records
len(planned[planned.PLANT_NAME.isna()].UNIT_ID.unique().tolist())

In [ ]:
len(unplanned[unplanned.PLANT_NAME.isna()].UNIT_ID.unique().tolist())

In [ ]:
len(planned[planned['PLANT_NAME'].isna()])

In [ ]:
planned[planned['PLANT_NAME'].isna()].pivot(index=old_cols,columns='OUTAGE_DATE',values='CAP_OFFLINE').fillna(0)

In [ ]:
# tars_named.pivot(index=['COUNTRY', 'OWNER_NAME', 'PLANT_NAME', 'UNIT_GROUP','UTYPE_DESC','UNIT_ID'],columns='OUTAGE_DATE',values='CAP_OFFLINE').fillna(0).to_csv('test_new_mapping.csv')
tars_named.pivot(index=['P_COUNTRY', 'OWNER_NAME', 'PLANT_NAME','UNIT_TYPE','UNIT_ID'],columns='OUTAGE_DATE',values='CAP_OFFLINE').fillna(0)